In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandasql as ps

In [22]:
#Aggregate events of matches to player level

path_map = f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/Statsbomb/open-data-master/data/events/"
num = 1

player_df = pd.DataFrame(columns=['match_id', 'player_id', 'player_name', 'passes_attempted', 'suc_passes', 'long_passes', 'suc_long_passes', 'passes_leading_to_shots ', 'passes_xg',
    'passes_leading_to_big_chance', 'passes_under_pressure', 'suc_passes_under_pressure', 'assists', 'interceptions', 'foul_committed',
    'yellow_card', 'red_card', 'shots_faced', 'saves_made', 'punches_made', 'punches_made_to_safe_place', 'dribbled_past',
    'miscontrol', 'shots', 'xgoal', 'shots_on_target', 'goals', 'non_penalty_goals', 'blocks', 'fouls_won', 'carries', 'progressive_carries',
    'dribbles', 'suc_dribbles', 'tackles', 'suc_tackles', 'aerial_lost', 'dispossessed', 'ball_recovery', 'clearance', 'error'])

for f in os.listdir(path_map):
    with open(path_map + f) as file:
        korger = json.load(file)


    df = pd.json_normalize(korger, sep='_').assign(match_id=f[:-5])

    df = df[df.player_name.isnull() == False]

    players = list(set(list(df.player_name)))

    for player in players:    
        current_player_df = df[df.player_name == player]

        match_id = current_player_df.match_id.iloc[0]
        player_id = current_player_df.player_id.iloc[0]
        player_name = current_player_df.player_name.iloc[0]

        #all_passes, suc_passes, suc_pass_rate
        pass_df = current_player_df[current_player_df.type_name == 'Pass']
        passes_attempted = len(pass_df)
        suc_passes = len(pass_df[pass_df.pass_outcome_name.isnull()])
        #suc_pass_rate = round(suc_passes/passes_attempted, 3)

        #long_passes, suc_long_passes, suc_long_pass_rate
        long_pass_df  = pass_df[pass_df.pass_length >= 25]
        long_passes = len(long_pass_df)
        suc_long_passes = len(long_pass_df[long_pass_df.pass_outcome_name.isnull()])
        #suc_long_pass_rate = round(suc_long_passes/long_passes, 3)

        #passes_leading_to_shots, passes_xg, passes_leading_to_big_chance
        pass_assisted_shots = pass_df[pass_df.pass_assisted_shot_id.isnull() == False]
        passes_leading_to_shots = len(pass_assisted_shots)
        passes_xg = 0
        passes_leading_to_big_chance = 0
        for shot in pass_assisted_shots.pass_assisted_shot_id:
            shot_xg = df[df.id == shot].shot_statsbomb_xg.values[0]
            passes_xg += shot_xg
            if shot_xg >= 0.6:
                passes_leading_to_big_chance += 1

        #pass_suc_rate_under_pressure
        under_pressure_df = pass_df[pass_df.under_pressure == True]
        passes_under_pressure = len(under_pressure_df)
        suc_passes_under_pressure = len(under_pressure_df[under_pressure_df.pass_outcome_name.isnull()])
        #suc_pass_rate_under_pressure = round(passes_under_pressure/suc_passes_under_pressure, 3)

        #assists
        try:
            assists = len(pass_df[pass_df.pass_goal_assist == True]) 
        except:
            assists = ''

        #interceptions
        interceptions = len(current_player_df[current_player_df.interception_outcome_name.isnull() == False])

        #foul committed, yellow card. red card
        foul_committed = len(current_player_df[current_player_df.type_name == 'Foul Committed'])
        try:
            yellow_card = len(current_player_df[current_player_df.foul_committed_card_name == 'Yellow Card'])
            red_card = len(current_player_df[current_player_df.foul_committed_card_name == 'Red Card'])
        except:
            yellow_card = ''
            red_card = ''
        
        #shot_faced, saves_made, save_rate, punches_made, punches_made_to_safe_place_rate
        goals_conceded = len(current_player_df[current_player_df['goalkeeper_type_name'] == 'Goal Conceded'])
        saves_made = len(current_player_df[current_player_df['goalkeeper_type_name'] == 'Shot Saved'])
        shots_faced =  goals_conceded + saves_made
        #save_rate = round(saves_made/shots_faced, 3)
        punches_made = len(current_player_df[current_player_df['goalkeeper_type_name'] == 'Punch'])
        punches_made_to_safe_place = len(current_player_df[current_player_df['goalkeeper_outcome_name'] == 'In Play Safe'])
        try:
            punches_made_to_safe_place_rate = round(punches_made_to_safe_place/punches_made, 3)
        except:
            punches_made_to_safe_place_rate = 0

        #dribbled_past
        dribbled_past = len(current_player_df[current_player_df['type_name'] == 'Dribbled Past'])

        #Miscontrol
        miscontrol = len(current_player_df[current_player_df['type_name'] == 'Miscontrol'])

        #shots, xgoal, shot_on_target, goal, shot_on_target_rate, non_penalty_goal, big chance???
        shots = len(current_player_df[current_player_df['type_name'] == 'Shots'])
        xgoal = current_player_df.shot_statsbomb_xg.sum()
        saved_shots = len(current_player_df[current_player_df.shot_outcome_name == 'Saved'])
        goals = len(current_player_df[current_player_df.shot_outcome_name == 'Goal'])
        shots_on_target = saved_shots + goals
        # try:
        #     shot_on_target_rate = round(shot_on_target/shots, 3)
        # except:
        #     shot_on_target_rate = 0
        non_penalty_goals = goals - len(current_player_df[current_player_df.shot_type_name == 'Penalty'])

        #blocks
        blocks = len(current_player_df[current_player_df['type_name'] == 'Block'])

        #foul won
        fouls_won = len(current_player_df[current_player_df['type_name'] == 'Foul Won'])

        #pressure
        fouls_won = len(current_player_df[current_player_df['type_name'] == 'Pressure'])

        #carries, avg_dist_carries
        carries = len(current_player_df[current_player_df['type_name'] == 'Carry'])
        carry_df = current_player_df[current_player_df.type_name == 'Carry'][['location', 'carry_end_location']]
        diff = []
        for i in range(len(carry_df)):
            diff.append(np.array(carry_df['location'].values[i]) - np.array(carry_df['carry_end_location'].values[i]))
        distance = 0
        distances = []
        for i in diff:
            distance += i[0]
            distances.append(i[0])
        try:    
            avg_dist_carries = distance/len(diff)
        except:
            avg_dist_carries = 0
        progressive_carries = 0
        for i in distances:
            if i >= 10:
                progressive_carries += i

        #dribbles, suc_dribbles, suc_dribbles_rate
        dribbles = len(current_player_df[current_player_df['type_name'] == 'Dribble'])
        suc_dribbles = len(current_player_df[current_player_df['dribble_outcome_name'] == 'Complete'])
        # try:
        #     suc_dribbles_rate = round(suc_dribbles/dribbles, 3)
        # except:
        #     suc_dribbles_rate = 0

        #tackles, suc_tackles, suc_tackles_rate, aerial_lost
        tackles = len(current_player_df[current_player_df['duel_type_name'] == 'Tackle'])
        suc_in_play = len(current_player_df[current_player_df.duel_outcome_name == 'Success In Play'])
        won = len(current_player_df[current_player_df.duel_outcome_name == 'Won'])
        suc_tackles = suc_in_play + won
        # try:
        #     suc_tackles_rate = round(suc_tackles/tackles, 3)
        # except:
        #     suc_tackles_rate = 0
        aerial_lost = len(current_player_df[current_player_df['duel_type_name'] == 'Aerial Lost'])

        #Dispossessed	
        dispossessed = len(current_player_df[current_player_df['duel_type_name'] == 'Dispossessed'])
        #dispossessed_own_half = len(current_player_df[current_player_df['duel_type_name'] == 'Dispossessed'])

        #Ball Recovery
        try:
            ball_recovery = len(current_player_df[current_player_df['duel_type_name'] == 'Ball Recovery'])
        except:
            bal_recovery = ''
        #ball_recovery_opp_half = len(current_player_df[current_player_df['duel_type_name'] == 'Ball Recovery'])

        #clearance
        clearance = len(current_player_df[current_player_df['duel_type_name'] == 'Clearance'])

        #error
        error = len(current_player_df[current_player_df['duel_type_name'] == 'Error'])

        row_to_add = np.array([match_id, player_id, player_name, passes_attempted, suc_passes, long_passes, suc_long_passes, passes_leading_to_shots , passes_xg,
        passes_leading_to_big_chance, passes_under_pressure, suc_passes_under_pressure, assists, interceptions, foul_committed,
        yellow_card, red_card, shots_faced, saves_made, punches_made, punches_made_to_safe_place, dribbled_past,
        miscontrol, shots, xgoal, shots_on_target, goals, non_penalty_goals, blocks, fouls_won, carries, progressive_carries,
        dribbles, suc_dribbles, tackles, suc_tackles, aerial_lost, dispossessed, ball_recovery, clearance, error])

        data_to_append = {}
        for i in range(len(player_df.columns)):
            data_to_append[player_df.columns[i]] = row_to_add[i]
        player_df = player_df.append(data_to_append, ignore_index = True)

    print(f'{num} / 1107')
    num += 1

1 / 1107
2 / 1107
3 / 1107
4 / 1107
5 / 1107
6 / 1107
7 / 1107
8 / 1107
9 / 1107
10 / 1107
11 / 1107
12 / 1107
13 / 1107
14 / 1107
15 / 1107
16 / 1107
17 / 1107
18 / 1107
19 / 1107
20 / 1107
21 / 1107
22 / 1107
23 / 1107
24 / 1107
25 / 1107
26 / 1107
27 / 1107
28 / 1107
29 / 1107
30 / 1107
31 / 1107
32 / 1107
33 / 1107
34 / 1107
35 / 1107
36 / 1107
37 / 1107
38 / 1107
39 / 1107
40 / 1107
41 / 1107
42 / 1107
43 / 1107
44 / 1107
45 / 1107
46 / 1107
47 / 1107
48 / 1107
49 / 1107
50 / 1107
51 / 1107
52 / 1107
53 / 1107
54 / 1107
55 / 1107
56 / 1107
57 / 1107
58 / 1107
59 / 1107
60 / 1107
61 / 1107
62 / 1107
63 / 1107
64 / 1107
65 / 1107
66 / 1107
67 / 1107
68 / 1107
69 / 1107
70 / 1107
71 / 1107
72 / 1107
73 / 1107
74 / 1107
75 / 1107
76 / 1107
77 / 1107
78 / 1107
79 / 1107
80 / 1107
81 / 1107
82 / 1107
83 / 1107
84 / 1107
85 / 1107
86 / 1107
87 / 1107
88 / 1107
89 / 1107
90 / 1107
91 / 1107
92 / 1107
93 / 1107
94 / 1107
95 / 1107
96 / 1107
97 / 1107
98 / 1107
99 / 1107
100 / 1107
101 / 11

In [23]:
player_df

,match_id,player_id,player_name,passes_attempted,suc_passes,long_passes,suc_long_passes,passes_leading_to_shots,passes_xg,passes_leading_to_big_chance,...,progressive_carries,dribbles,suc_dribbles,tackles,suc_tackles,aerial_lost,dispossessed,ball_recovery,clearance,error
0,2275050,19780.0,Rachel Rowe,71,45,12,5,1,0.035531893,0,...,52.900000000000006,2,1,5,4,2,0,0,0,0
1,2275050,10251.0,Fara Williams,45,23,10,1,3,0.7580202300000001,0,...,0,4,2,2,2,1,0,0,0,0
2,2275050,18149.0,Remi Allen,40,25,13,6,2,0.095943831,0,...,0,5,2,5,5,3,0,0,0,0
3,2275050,19500.0,Sarah Emma Mayling,39,25,11,4,0,0,0,...,0,1,1,3,3,2,0,0,0,0
4,2275050,31563.0,Rachel Williams,27,14,3,1,1,0.17673256,0,...,0,1,1,2,1,7,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30797,3795108,6892.0,Pau Francisco Torres,150,143,40,36,0,0,0,...,10.199999999999996,0,0,2,0,4,0,0,0,0
30798,3795108,16027.0,Kevin Mbabu,5,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30799,3795108,5550.0,Yann Sommer,58,36,45,23,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30800,3795108,5537.0,Fabian Lukas Schär,6,2,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [26]:
player_df.to_excel(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/Statsbomb/events.xlsx")

In [25]:
#Match information

path_map = [f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/Statsbomb/open-data-master/data/matches/11/",
    f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/Statsbomb/open-data-master/data/matches/16/",
    f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/Statsbomb/open-data-master/data/matches/72/",
    f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/Statsbomb/open-data-master/data/matches/43/",
    f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/Statsbomb/open-data-master/data/matches/37/",
    f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/Statsbomb/open-data-master/data/matches/55/",
    f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/Statsbomb/open-data-master/data/matches/49/2",
    f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/Statsbomb/open-data-master/data/matches/2/"]    

match_df = pd.DataFrame(columns=['match_id', 'competition_id','competition_name', 'season_id',
       'season_name', 'home_team_id',
       'home_team_name', 'home_team_gender', 'home_team_country_id',
       'home_team_country_name',  'away_team_id', 'away_team_name',
       'away_team_gender', 'away_team_country_id', 'away_team_country_name'])

num = 1
for folder in path_map:
    for f in os.listdir(folder):
        with open(folder + f) as file:
            data = json.load(file)
        
        df = pd.json_normalize(data, sep='_')

        try:
            match_id = df.match_id
            competition_id = df.competition_competition_id
            competition_name = df.competition_competition_name
            season_id = df.season_season_id
            season_name = df.season_season_name
            home_team_id = df.home_team_home_team_id
            home_team_name = df.home_team_home_team_name
            home_team_gender = df.home_team_home_team_gender
            home_team_country_id = df.home_team_country_id
            home_team_country_name = df.home_team_country_name
            away_team_id = df.away_team_away_team_id
            away_team_name = df.away_team_away_team_name
            away_team_gender = df.away_team_away_team_gender
            away_team_country_id = df.away_team_country_id
            away_team_country_name = df.away_team_country_name

            for i in range(len(match_id)):
                row_to_add = np.array([match_id[i], competition_id[i], competition_name[i], season_id[i],
                season_name[i], home_team_id[i],
                home_team_name[i], home_team_gender[i], home_team_country_id[i],
                home_team_country_name[i],  away_team_id[i], away_team_name[i],
                away_team_gender[i], away_team_country_id[i], away_team_country_name[i]])

                data_to_append = {}
                for i in range(len(match_df.columns)):
                    data_to_append[match_df.columns[i]] = row_to_add[i]
                match_df = match_df.append(data_to_append, ignore_index = True)
        except:
            pass
       
        print(f'{num} / 50')
        num += 1


1 / 50
2 / 50
3 / 50
4 / 50
5 / 50
6 / 50
7 / 50
8 / 50
9 / 50
10 / 50
11 / 50
12 / 50
13 / 50
14 / 50
15 / 50
16 / 50
17 / 50
18 / 50
19 / 50
20 / 50
21 / 50
22 / 50
23 / 50
24 / 50
25 / 50
26 / 50
27 / 50
28 / 50
29 / 50
30 / 50
31 / 50
32 / 50
33 / 50
34 / 50
35 / 50
36 / 50
37 / 50
38 / 50
39 / 50
40 / 50
41 / 50


In [26]:
match_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1096 entries, 0 to 1095
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   match_id                1096 non-null   object
 1   competition_id          1096 non-null   object
 2   competition_name        1096 non-null   object
 3   season_id               1096 non-null   object
 4   season_name             1096 non-null   object
 5   home_team_id            1096 non-null   object
 6   home_team_name          1096 non-null   object
 7   home_team_gender        1096 non-null   object
 8   home_team_country_id    1096 non-null   object
 9   home_team_country_name  1096 non-null   object
 10  away_team_id            1096 non-null   object
 11  away_team_name          1096 non-null   object
 12  away_team_gender        1096 non-null   object
 13  away_team_country_id    1096 non-null   object
 14  away_team_country_name  1096 non-null   object
dtypes: o

In [27]:
match_df.to_excel(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/Statsbomb/matches.xlsx")

In [6]:
#join player and match df
q1 = """SELECT DF.*,
        m.*

        FROM player_df DF 

        LEFT JOIN match_df m
        ON DF.match_id = m.match_id

        """

statsbomb_df = ps.sqldf(q1, locals())

In [7]:
statsbomb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30802 entries, 0 to 30801
Data columns (total 58 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    30802 non-null  int64  
 1   match_id                      30802 non-null  int64  
 2   player_id                     30802 non-null  float64
 3   player_name                   30802 non-null  object 
 4   passes_attempted              30802 non-null  int64  
 5   suc_passes                    30802 non-null  int64  
 6   long_passes                   30802 non-null  int64  
 7   suc_long_passes               30802 non-null  int64  
 8   passes_leading_to_shots       30802 non-null  int64  
 9   passes_xg                     30802 non-null  float64
 10  passes_leading_to_big_chance  30802 non-null  int64  
 11  passes_under_pressure         30802 non-null  int64  
 12  suc_passes_under_pressure     30802 non-null  int64  
 13  a

In [8]:
statsbomb_df.to_excel(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/Statsbomb/statsbomb.xlsx")